# Estimating Motors from a Variety of Geometric Data in 3D Conformal Geometric Algebra

Robert Valkenburg and Leo Dorst

In [177]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=2, suppress=True)
import versor as vsr
from motor_estimation_valkenburg_dorst import point_matrix, dual_line_matrix, dual_plane_matrix
from motor_estimation import MotorEstimationSolver

Populating the interactive namespace from numpy and matplotlib


In [178]:
import versor as vsr

## Dataset Generation

In [179]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor)

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]


In [180]:
n_points = 1000
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() 
            for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

In [181]:
lines_a = [vsr.Dll(vsr.Vec(*np.random.normal(0.0,0.8,3)),
                   vsr.Vec(*np.random.normal(0.0,0.8,3)))
          for i in range(1000)]
lines_b = [line.spin(motor) for line in lines_a]

In [182]:
n_planes = 10
planes_a = []
for i in range(10):
    dir_vec = np.random.normal(0.0, 0.8, 4)
    dir_vec = dir_vec / np.linalg.norm(dir_vec)
    distance = np.random.normal(0.0, 0.8,1)
    arr = dir_vec + distance
    planes_a.append(vsr.Dlp(dir_vec[0],dir_vec[1],dir_vec[2],distance[0]))
planes_b = [plane.spin(motor) for plane in planes_a]

In [183]:
class VDMotorEstimationSolver(object):
    def __init__(self):
        self._point_matrix = np.zeros((8,8))
        self._dual_line_matrix = np.zeros((8,8))
        self._dual_plane_matrix = np.zeros((8,8))
        
    @property
    def L(self):
        return self._point_matrix + self._dual_line_matrix + self._dual_plane_matrix
    
    def add_point_observations(self, ps, qs):
        for p, q in zip(ps,qs):
            self._point_matrix += point_matrix(p,q)
    
    def add_dual_line_observations(self,ps,qs):
        for p, q in zip(ps,qs):
            self._dual_line_matrix += dual_line_matrix(p,q)
    
    def add_dual_plane_observations(self, ps,qs):
        for p, q in zip(ps,qs):
            self._dual_plane_matrix += dual_plane_matrix(p,q)
        
    def solve(self):
        L = self.L
        Lrr = L[:4,:4]
        Lrq = L[:4,4:]
        Lqr = L[4:,:4]
        Lqq = L[4:,4:]
        Lp = Lrr - np.dot(Lrq, np.dot(np.linalg.pinv(Lqq), Lqr))
        w,v = np.linalg.eig(Lp)
        r = v[:,np.argmax(w)]
        q = np.dot(-np.dot(np.linalg.pinv(Lqq), Lqr),r)
        return vsr.Mot(*np.array([r,q]).ravel())  

In [184]:
vd = VDMotorEstimationSolver()
# vd.add_dual_plane_observations(planes_a, planes_b)
vd.add_dual_line_observations(lines_a, lines_b)
vd.add_point_observations(points_a, points_b_noisy)
vd_estimated_motor = vd.solve()
print(np.array(vd_estimated_motor))

[-0.87 -0.    0.5   0.    0.68  0.43  0.18  0.25]


In [185]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(points_a, points_b_noisy):
    mes.add_point_correspondences_residual_block(a,b)
for a, b in zip(lines_a, lines_b):
    mes.add_line_correspondences_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
mes.num_linear_solve_threads = 4
mes.num_threads = 4
# mes.function_tolerance = 1e-16
# mes.gradient_tolerance = 1e-16
# mes.parameter_tolerance = 1e-16
te_estimated_motor, summary = mes.solve()
print(summary['full_report'])
print(np.array(te_estimated_motor))


Solver Summary (v 1.12.0-eigen-(3.2.92)-lapack-suitesparse-(4.4.6)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                          2000                     2000
Residual                                 9000                     9000

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     4                        4
Linear solver threads                       4                        4

Cost:
Initial                          5.512107e+03
Final              

In [186]:
print(np.array(motor))

[ 0.87  0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]


In [187]:
np.allclose(vd_estimated_motor, te_estimated_motor, atol=1e-03)

False

In [188]:
print(vd.L)

[[ -918.8    -12.12 -2264.28     0.4   -983.9   -997.76 -1008.48     0.  ]
 [  -12.12 -4878.22   -14.49    36.98  1002.   -1019.24     0.   -1008.48]
 [-2264.28   -14.49 -3593.39   -34.4   1035.96     0.   -1019.24   997.76]
 [    0.4     36.98   -34.4  -4920.3      0.    1035.96 -1002.    -983.9 ]
 [ -983.9   1002.    1035.96     0.   -2000.       0.       0.       0.  ]
 [ -997.76 -1019.24     0.    1035.96     0.   -2000.       0.       0.  ]
 [-1008.48     0.   -1019.24 -1002.       0.       0.   -2000.       0.  ]
 [    0.   -1008.48   997.76  -983.9      0.       0.       0.   -2000.  ]]
